In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from IPython import display


from tqdm import tqdm_notebook

In [2]:
from data_loader import Preprocess
from model import TextGAN

Using TensorFlow backend.


In [3]:
params = {'raw_text': 'raw_text.txt', 'train_file': 'train_data.tsv', 'test_file': 'test_data.tsv',
          'embedding': 'embedding.fasttext', 'batch_size': 1, 'epoch': 10000, 'max_seq_len': 5,
          'embedding_size': 256, 'plt_frq': 20}

In [4]:
processor = Preprocess(raw_text = params['raw_text'], max_seq_len=params['max_seq_len'])
processor.process(params['train_file'], mode='train')
processor.process(params['test_file'], mode='test')

Vocabulary Size: 28461


100%|██████████| 500/500 [00:00<00:00, 2329.93it/s]


train_data.tsv processed.


100%|██████████| 500/500 [00:00<00:00, 3102.64it/s]

test_data.tsv processed.


In [5]:
# if params['embedding']:
#     processor.load_embedding(params['embedding'], params['embedding_size'])

In [6]:
params['vocab_size'] = processor.vocab_size

In [7]:
text_gan = TextGAN(vocabulary=params['vocab_size'], max_seq_len=params['max_seq_len'],
                embedding_size=params['embedding_size'])

In [8]:
text_gan.build_gan()

In [ ]:
# print('\nGENERATOR: ')
# text_gan.generator.summary()

In [ ]:
# print('\nDISCRIMINATOR: ')
# text_gan.discriminator.summary()

In [ ]:
text_gan.discriminator.inputs, text_gan.discriminator.outputs

([<tf.Tensor 'sentence_input:0' shape=(?, 5, 28461) dtype=float32>,
  <tf.Tensor 'paraphrase_input:0' shape=(?, 5, 28461) dtype=float32>],
 [<tf.Tensor 'dense_1/Softmax:0' shape=(?, 2) dtype=float32>])

In [ ]:
text_gan.generator.inputs, text_gan.generator.outputs

([<tf.Tensor 'sentence_input_1:0' shape=(?, 5, 28461) dtype=float32>],
 [<tf.Tensor 'dense_2/truediv:0' shape=(?, 5, 28461) dtype=float32>])

In [ ]:
def plot_loss(losses):
        display.clear_output(wait=True)
        display.display(plt.gcf())
        plt.figure(figsize=(10,8))
        plt.plot(losses["d_loss"], label='discriminitive loss')
        plt.plot(losses["d_acc"], label='discriminitive accuracy')
        
        plt.plot(losses["g_loss"], label='generative loss')
        plt.legend()
        plt.show()

def plot_gen(epoch=1):
    sentences, _ = processor.posotive_batch(params['batch_size'])
    output = text_gan.generator.predict(sentences)
    generated_paraphrases = output[1]
    
    with open('generated/paraphrases_{}.txt'.format(epoch), 'w') as f:
        for i in range(len(sentences)):
            original_sentence = processor.convert_to_sentence(sentences[i])
            generated_paraphrase = processor.convert_to_sentence(np.argmax(generated_paraphrases[i], axis=2))
            
            f.write('{} | {}\n'.format(original_sentence, generated_paraphrase))    

In [ ]:
losses = {'d_loss': [], 'g_loss': [], 'd_acc': []}

In [ ]:
def get_label(batch_size, mode='real'):
    label = []
    for _ in range(batch_size):
        if mode == 'real':
            label.append([1, 0])
        else:
            label.append([0, 1])
            
    return np.array(label)

In [ ]:
for epoch in range(params['epoch']):
    sentence, paraphrase = processor.positive_batch(params['batch_size'])
    
    gen_sentences = text_gan.generator.predict(sentence)
    
    real_input = [sentence, paraphrase]
    fake_input = [sentence, gen_sentences]
    
    d_loss_real = text_gan.discriminator.train_on_batch(real_input, get_label(len(real_input[0]), mode='real'))
    print('Real Trained')
    d_loss_fake = text_gan.discriminator.train_on_batch(fake_input, get_label(len(fake_input[0]), mode='fake'))
    print('Fake Trained')
    
    d_loss = 0.5*np.add(d_loss_real, d_loss_fake)
    losses['d_loss'].append(d_loss[0])
    losses['d_acc'].append(d_loss[1])
    
    # Generator
    sentence, _ = processor.positive_batch(params['batch_size'])
    g_loss = text_gan.text_gan.train_on_batch(sentence, get_label(len(sentence), mode='real'))
    losses['g_loss'].append(g_loss)
    
    if epoch%params['plt_frq'] == params['plt_frq'] - 1:
        plot_loss(losses)
        save_gen(epoch=epoch)
        
    break

/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:478: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
